# Team Information

Project Title: Data Analsyis on the Pokemon Pokedex

Team Name: Team 6

Member Names w/ PIDs: Rohan Jaggannagari (rjaggannagari), Jayanth Kalidindi (jayk21), Luke Hylton (lnhylton), Markus Homann (homann), Aniket Sonnakul (anikets)

## Project Introduction

Description: In this project, we intend to scrape data from the internet on the Pokemon Pokedex and get a each Pokemon's weaknesses, strengths, statistics, and abilties. From there, we will merge additional data from the internet on specfic Pokemon to give us more information to work on. By the end of this project, we will be able to know which Pokemon types are the strongest, which abilties correlate with the best statistics, best Pokemon per type with/without legendary Pokemon. This information is useful since many of the digital Pokemon games are based on this information and doing data analysis on it will be useful to users to know which Pokemon fits their playstyles and strengths.

Potential Research Questions: // Fill out later

Potential Source Data: https://pokemondb.net/, https://www.serebii.net/pokemongo/pokemon/, https://db.pokemongohub.net/, https://www.kaggle.com/code/mmetter/pokemon-data-analysis-tutorial/input?select=pokemon.csv, https://www.kaggle.com/datasets/rounakbanik/pokemon?datasetId=2756